In [ ]:
print("hello world")

: 

In [ ]:
from petls import PersistentSheafLaplacian, sheaf_simplex_tree, Complex
# from Bio.PDB import PDBParser
# from preprocess import get_ligand_data, read_pdb_biopython
import pandas as pd
import gudhi as gd
import numpy as np
import os

: 

In [ ]:
def atom_distance(a, b):
    # also known as EUC
    return np.sqrt(np.power((a[0] - b[0]), 2.0)
                   + np.power((a[1] - b[1]), 2.0)
                   + np.power((a[2] - b[2]), 2.0))

def filter_heavy_atoms(df, include_list):
    return df[df.element.isin(include_list)]


def add_radii(df, elts, radii):
    element_df = pd.DataFrame(list(zip(elts, radii)), columns=["element", "r"])
    df = df.merge(element_df, on="element")
    return df

In [ ]:
pdbid = "1E68"
from readin import mol2_to_pqre
protein = mol2_to_pqre(pdbid, "charged", base_dir="../data/misc")




filtered_pro = np.array(filter_heavy_atoms(protein,["C"]))

combined = filtered_pro[:,[0,1,2,4]]
# return combined

In [ ]:
def get_extra_data(combined):
    extra_data = {}
    for i, vertex in enumerate(combined):
        extra_data[tuple([i])] = [*vertex]  # store x,y,z,r,element,charge for each vertex
        print(vertex)
    return extra_data

In [ ]:
def min_nonzero(X,tol=1e-3):
    nonzeros = [s for s in X if s > tol]
    if len(nonzeros) == 0:
        return 0
    return min(nonzeros)

In [ ]:
import petls
def my_restriction(simplex: list[int], coface: list[int], sst: petls.sheaf_simplex_tree) -> float:
    from math import sqrt
    if len(simplex) == 1:
        if simplex == [coface[0]]:
            sibling = [coface[1]]
        else:
            sibling = [coface[0]]
        
        coords_simplex = sst.extra_data[tuple(simplex)][0:3]
        coords_sibling = sst.extra_data[tuple(sibling)][0:3]
        distance = sqrt((coords_simplex[0] - coords_sibling[0])**2 \
                    + (coords_simplex[1] - coords_sibling[2])**2 \
                    + (coords_simplex[2] - coords_sibling[1])**2)
        return sst.extra_data[tuple(sibling)][3] / distance # charge / distance
    elif len(simplex) == 2:
        coeff = 1.0
        for (sibling, _) in sst.st.get_boundaries(coface):
            if sibling == simplex:
                opposite_vertex = coface[sst.coface_index(simplex,coface)]
                coeff = coeff * sst.extra_data[tuple([opposite_vertex])][3] #charge
            else:
                coeff = coeff / sst.st.filtration(sibling)
        return coeff

In [ ]:
from gudhi import AlphaComplex
alpha_complex = AlphaComplex(points=combined[:,0:3])
simplex_tree = alpha_complex.create_simplex_tree()
simplex_tree.prune_above_dimension(2)
# simplex_tree.prune_above_filtration(4.0)


In [ ]:
dgms = simplex_tree.persistence()

dgm1 = simplex_tree.persistence_intervals_in_dimension(1)
dgm1_unsquared = np.sqrt(dgm1)
print(f"Number of 1D features: {len(dgm1)}")
print(dgm1_unsquared)

In [ ]:
gd.plot_persistence_diagram(dgms)

In [ ]:
all_filt = np.arange(0.0,8.0,0.1)
print(all_filt)

extra_data = get_extra_data(combined)
print(extra_data)
sst = sheaf_simplex_tree(simplex_tree, extra_data, my_restriction)

psl = PersistentSheafLaplacian(sst)
dims = [0,1]
request_list = []
for dim in dims:
    request_list+= [(dim, filt,filt+0.4) for filt in all_filt]
print(request_list)

In [ ]:
from math import sqrt
spectra = psl.spectra(request_list=request_list)


In [ ]:
print("hello world")

: 

In [ ]:
tol = 1e-16
dim0_spectra = [[filt, min_nonzero(spectrum,tol), len([s for s in spectrum if s < tol])] for (dim, filt, _, spectrum) in spectra if dim == 0]
dim1_spectra = [[filt, min_nonzero(spectrum,tol), len([s for s in spectrum if s < tol])] for (dim, filt, _, spectrum) in spectra if dim == 1]

In [ ]:
print(dim0_spectra.shape)

In [ ]:
dim0_spectra

In [ ]:
import matplotlib.pyplot as plt
d0 = np.array(dim0_spectra)
d1 = np.array(dim1_spectra)
d = d1
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(d[:,0],d[:,1],color='b')
ax2.plot(d[:,0],d[:,2],color='r')
plt.xlim(0,8)
plt.hlines(y=tol,xmin=0,xmax=8,label="tol",linestyles=":")
ax1.legend(loc="upper right")
ax1.set_yscale("log")
ax2.legend(loc = "upper left")
# ax1.set_ylim(-0.001, 0.05)
# ax2.set_ylim(-0.001, 200)
# ax1.set_ylim(-0.001,0.035)
# ax2.set_ylim(-0.001,401)

Try trapezoid

In [ ]:
def get_extra_data_2d(combined):
    extra_data = {}
    for i, vertex in enumerate(combined):
        extra_data[tuple([i])] = [*vertex]  # store x,y,charge for each vertex
        # print(vertex)
    return extra_data

In [ ]:
from math import sqrt
trapezoid = np.array([[1/4,sqrt(15)/4,-1],
             [3/4,sqrt(15)/4,1],
             [0,0,1],
             [1,0,1]])

alpha_complex2 = AlphaComplex(points=trapezoid[:,0:2])
simplex_tree2 = alpha_complex2.create_simplex_tree()
simplex_tree2.num_simplices()
# simplex_tree2.prune_above_dimension(2)
# simplex_tree2.prune_above_filtration(2.0)

In [ ]:
extra_data = get_extra_data_2d(trapezoid)
print(extra_data)
sst = sheaf_simplex_tree(simplex_tree2, extra_data, my_restriction_2d)

psl = PersistentSheafLaplacian(sst)
dims = [0,1]
request_list = []
for dim in dims:
    request_list+= [(dim, filt,filt+0.2) for filt in all_filt]
print(request_list)

In [ ]:
import petls
def F(simplex: list[int], sst: petls.sheaf_simplex_tree):
    if len(simplex) == 1:
        return 1.0
    elif len(simplex) == 2:
        v0 = sst.extra_data[tuple([simplex[0]])][0:2]
        v1 = sst.extra_data[tuple([simplex[1]])][0:2]
        return sqrt((v0[0] - v1[0])**2 + (v0[1] - v1[1])**2) # length
    elif len(simplex) == 3:
        v0 = sst.extra_data[tuple([simplex[0]])][0:2]
        v1 = sst.extra_data[tuple([simplex[1]])][0:2]
        v2 = sst.extra_data[tuple([simplex[2]])][0:2]
        a = sqrt((v0[0] - v1[0])**2 + (v0[1] - v1[1])**2)
        b = sqrt((v1[0] - v2[0])**2 + (v1[1] - v2[1])**2)
        c = sqrt((v2[0] - v0[0])**2 + (v2[1] - v0[1])**2)
        return a*b*c

In [ ]:
import petls

    
def my_restriction_2d(simplex: list[int], coface: list[int], sst: petls.sheaf_simplex_tree) -> float:
    from math import sqrt
    # return 1
    complement_simplex = list(set(coface)-set(simplex)) 
    assert len(complement_simplex) == 1
    # print("complement_simplex:", complement_simplex)
    # print("extra_data[tuple(complement_simplex)]:", extra_data[tuple(complement_simplex)])
    q_j = extra_data[tuple(complement_simplex)][2]
    return F(simplex,sst) * q_j / F(coface, sst)        
    #     if simplex == [coface[0]]:
    #         sibling = [coface[1]]
    #     else:
    #         sibling = [coface[0]]
        
    #     coords_simplex = sst.extra_data[tuple(simplex)][0:3]
    #     coords_sibling = sst.extra_data[tuple(sibling)][0:3]
    #     distance = sqrt((coords_simplex[0] - coords_sibling[0])**2 \
    #                 + (coords_simplex[1] - coords_sibling[2])**2 \
    #                 + (coords_simplex[2] - coords_sibling[1])**2)
    #     return sst.extra_data[tuple(sibling)][3] / distance # charge / distance
    # elif len(simplex) == 2:
    #     coeff = 1.0
    #     for (sibling, _) in sst.st.get_boundaries(coface):
    #         if sibling == simplex:
    #             opposite_vertex = coface[sst.coface_index(simplex,coface)]
    #             coeff = coeff * sst.extra_data[tuple([opposite_vertex])][3] #charge
    #         else:
    #             coeff = coeff / sst.st.filtration(sibling)
    #     return coeff

In [ ]:
sst = sheaf_simplex_tree(simplex_tree2, extra_data, my_restriction_2d)


In [ ]:
psl = PersistentSheafLaplacian(sst)

In [ ]:
# all_filt = psl.get_all_filtrations()
all_filt = np.arange(0.0,1.0,0.01)
print(all_filt)

In [ ]:
dims = [0,1]
request_list = []
for dim in dims:
    request_list+= [(dim, filt,filt+0.2) for filt in all_filt]
print(request_list)

In [ ]:
spectra = psl.spectra(request_list=request_list)
dim0_spectra = [[sqrt(filt), min_nonzero(spectrum), len([s for s in spectrum if s < 1e-2])] for (dim, filt, _, spectrum) in spectra if dim == 0]
dim1_spectra = [[sqrt(filt), min_nonzero(spectrum), len([s for s in spectrum if s < 1e-2])] for (dim, filt, _, spectrum) in spectra if dim == 1]

In [ ]:
print(dim0_spectra)
print(dim1_spectra)

In [ ]:
import matplotlib.pyplot as plt
d0 = np.array(dim0_spectra)
d1 = np.array(dim1_spectra)

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(d1[:,0],d1[:,1],color='g')
ax2.plot(d1[:,0],d1[:,2],color='b')
plt.xlim(0,1)
ax1.set_ylim(-0.1,8.1)
ax2.set_ylim(-0.1,1.1)